<a href="https://colab.research.google.com/github/AUT-Student/CN-HW1/blob/main/ComplexNetwork_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Edge():
  source: int
  destination: int

  def sort(self):
    return Edge(min(self.destination, self.source),
                max(self.destination, self.source))
    
  def __eq__(self, other):
    return (self.source == other.source and self.destination == other.destination) or\
           (self.destination == other.source and self.source == other.destination)


In [41]:
class Graph():
  def __init__(self):
    self.nodes = set()
    self.edges = set()
    self.edge_list = dict()

  def add_node(self, node):
    self.nodes.add(node)

    if node not in self.edge_list:
      self.edge_list[node] = set()

  def add_edge(self, edge:Edge):
    self.add_node(edge.source)
    self.add_node(edge.destination)

    self.edge_list[edge.source].add(edge.destination)
    self.edge_list[edge.destination].add(edge.source)
    self.edges.add(edge)

  def remove_edge(self, edge:Edge):
    self.edge_list[edge.source].remove(edge.destination)
    self.edge_list[edge.destination].remove(edge.source)
    self.edges.remove(edge)

  def is_exist(self, edge:Edge):
    return edge in self.edges

  def get_number_nodes(self):
    return len(self.nodes)

  def get_number_edges(self):
    return len(self.edges)

In [42]:
import random

class ErdosRenyiGraph(Graph):
  def __init__(self, number_nodes, number_edges):
    super().__init__()
    self.number_nodes = number_nodes
    self.number_edges = number_edges

  def generate(self):
    for i in range(self.number_nodes):
      self.add_node(i)

    all_possible_edges = []
    for node1 in range(self.number_nodes):
      for node2 in range(node1+1, self.number_nodes):
        all_possible_edges.append(Edge(node1, node2))
    
    selected_edges = random.sample(all_possible_edges, self.number_edges)
    for edge in selected_edges:
      self.add_edge(edge)


In [43]:
er_graph = ErdosRenyiGraph(number_nodes=7624, number_edges=27806)

In [44]:
er_graph.generate()

In [45]:
er_graph.get_number_edges()

27806

In [55]:
import random

class SmallWorldGraph(Graph):
  def __init__(self, number_nodes, number_edges, random_prob):
    super().__init__()
    self.number_nodes = number_nodes
    self.number_edges = number_edges
    self.scale = self.number_edges / self.number_nodes
    self.random_prob = random_prob

  def random_change(self, number_random):
    remove_edges = random.sample(self.edges, number_random)
    source_edges_remove = list()
    for edge in remove_edges:
      self.remove_edge(edge)
      source_edges_remove.append(edge.source)

    source_edges_remove = random.sample(source_edges_remove, self.number_edges - self.get_number_edges())

    all_nodes = list(self.nodes)
    for node1 in source_edges_remove:
      while True:
        node2 = random.choice(all_nodes)
        edge = Edge(node1, node2)
        if node2 != node1 and not self.is_exist(edge):
          self.add_edge(edge)
          break

  def generate(self):
    for i in range(self.number_nodes):
      self.add_node(i)

    number_edges1 = int(self.scale)
    number_edges2 = int((self.scale - int(self.scale))*100 + 1)
    
    counter1 = 0
    counter2 = 0
    
    for node1 in range(self.number_nodes):
      for distance in range(1, number_edges1+1):
        node2 = (node1 + distance)%self.number_nodes
        self.add_edge(Edge(node1, node2))
        counter1 += 1

      if (node1 %100) < number_edges2:
        node2 = (node1 + number_edges1 + 1)%self.number_nodes
        self.add_edge(Edge(node1, node2))
        counter2 += 1

    print(f"Number Edges = {self.get_number_edges()}")

    number_random = int((self.get_number_edges() - self.number_edges) +\
                          self.random_prob * self.get_number_edges())
    
    print(f"number_random = {number_random}")

    self.random_change(number_random = number_random)

    print(f"Number Edges = {self.get_number_edges()}")

In [56]:
small_world_graph = SmallWorldGraph(number_nodes=7624, number_edges=27806, random_prob=0.05)

In [57]:
small_world_graph.generate()

Number Edges = 27836
number_random = 1421
Number Edges = 27806


In [37]:
random.choice(set([1,4,5,6]))

TypeError: ignored

In [56]:
27806/7624

3.6471668415529908

In [36]:
random.sample([2,4,6,8,10,12], 3)

[12, 6, 2]

In [ ]:
import random

random.choice([1,2,3,4])

In [26]:
for i in range(5):
  for j in range(i, 5):
    print(f"i = {i}, j = {j}")

i = 0, j = 0
i = 0, j = 1
i = 0, j = 2
i = 0, j = 3
i = 0, j = 4
i = 1, j = 1
i = 1, j = 2
i = 1, j = 3
i = 1, j = 4
i = 2, j = 2
i = 2, j = 3
i = 2, j = 4
i = 3, j = 3
i = 3, j = 4
i = 4, j = 4


In [16]:
!gdown https://snap.stanford.edu/data/lastfm_asia.zip

Downloading...
From: https://snap.stanford.edu/data/lastfm_asia.zip
To: /content/lastfm_asia.zip
100% 6.53M/6.53M [00:00<00:00, 8.88MB/s]


In [17]:
!unzip /content/lastfm_asia.zip

Archive:  /content/lastfm_asia.zip
replace lasftm_asia/lastfm_asia_edges.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [18]:
import pandas as pd

lastfm_dataset = pd.read_csv("/content/lasftm_asia/lastfm_asia_edges.csv")

In [21]:
lastfm_graph = Graph()

for i, data in lastfm_dataset.iterrows():
  node1 = data["node_1"]
  node2 = data["node_2"]

  lastfm_graph.add_edge(Edge(node1, node2))

In [22]:
lastfm_graph.number_nodes()

7624

In [23]:
lastfm_graph.number_edges()

27806.0

In [3]:
!head /content/lasftm_asia/lastfm_asia_edges.csv

node_1,node_2
0,747
1,4257
1,2194
1,580
1,6478
1,1222
1,5735
1,7146
1,2204


In [5]:
!head /content/lasftm_asia/lastfm_asia_target.csv

id,target
0,8
1,17
2,3
3,17
4,5
5,17
6,3
7,6
8,0


In [4]:
!head /content/lasftm_asia/lastfm_asia_features.json

{"0": [2964, 3900, 3902, 2402, 6185, 509, 7627, 3389, 2407, 5, 4403, 3633, 5875, 3395, 3531, 6908, 3202, 5883, 21, 7389, 1449, 5391, 3400, 2911, 6618, 3315, 1454, 4904, 3409, 3920, 6401, 3923, 4909, 2919, 532, 1463, 540, 4427, 5907, 7412, 1926, 7416, 3935, 5913, 4349, 6420, 1472, 6948, 3979, 58, 5918, 5920, 5417, 6950, 4439, 1020, 4442, 5925, 1945, 3947, 3442, 5941, 1027, 4805, 5528, 3455, 2457, 4159, 7235, 5720, 1359, 7444, 2463, 1044, 4461, 3042, 3471, 1504, 6977, 7450, 1507, 4464, 6388, 584, 6454, 585, 3479, 6455, 6986, 4893, 3154, 3482, 3483, 2477, 6041, 6731, 1138, 2977, 6468, 3993, 2006, 7366, 7005, 602, 3998, 2991, 5460, 7009, 7010, 3500, 2499, 6736, 6367, 5464, 7014, 4990, 7016, 2999, 1530, 4500, 1075, 338, 5468, 1533, 1078, 2541, 7024, 6635, 3523, 4019, 2024, 4081, 498, 5011, 3069, 3530, 4025, 1549, 133, 7037, 3533, 1552, 5489, 2533, 2581, 5025, 4374, 7569, 4383, 4262, 146, 5037, 2055, 5649, 5502, 661, 32, 4041, 667, 7530, 2552, 6055, 2657, 2430, 5233, 4497, 5158, 3562, 4546, 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



977, 7452, 4464, 6375, 2963, 3970, 5445, 1984, 5943, 3480, 4967, 2475, 6202, 2477, 6296, 4472, 6731, 2478, 2975, 2480, 4698, 3988, 6464, 6995, 1062, 3992, 7465, 4350, 2487, 3496, 7469, 4981, 5962, 4487, 7007, 5461, 605, 7009, 7777, 7012, 7475, 4988, 7478, 7014, 4990, 7694, 7481, 7005, 1450, 3400, 2506, 3510, 5698, 3552, 4996, 5470, 6299, 1300, 6765, 2991, 4460, 4079, 2514, 3523, 5476, 5479, 7029, 1090, 629, 992, 132, 7474, 2526, 3698, 5015, 2528, 7444, 7038, 4029, 4522, 2532, 5022, 5025, 2832, 5998, 2538, 3718, 3539, 1106, 4351, 2050, 2499, 2543, 5920, 3546, 6519, 3044, 3920, 436, 7060, 6924, 6053, 2061, 7065, 1118, 5970, 5507, 1573, 5464, 2555, 156, 441, 4540, 2014, 3321, 5233, 5320, 3053, 4991, 6533, 1581, 3558, 2752, 4546, 678, 4059, 4061, 7406, 4547, 3057, 7081, 5524, 2083, 349, 3062, 6039, 4071, 690, 4557, 4418, 6041, 5061, 7104, 694, 7089, 3928, 6047, 5067, 4167, 7095, 180, 2071, 1599, 5546, 6055, 2105, 4092, 1603, 711, 3579, 5077, 1156, 189, 4269, 6554, 4266, 1604, 4997, 4096, 2